# Make plots for the RelativisticOrthogonalization project

In [ ]:
import numpy as np
import pylab as plt
import matplotlib as mpl

In [ ]:
# plotting defaults
mpl.rcParams['axes.xmargin'] = 0
mpl.rcParams['axes.ymargin'] = 0
mpl.rcParams['figure.figsize'] = (3, 3)
dpi=200

In [ ]:
# Start with Orthogonal Group
d = 2
METRIC = np.eye(d)

In [ ]:
def inner(v1, v2):
    return v1.T @ METRIC @ v2.T

def norm(v):
    return np.sqrt(np.abs(inner(v, v)))

def orthogonalize(vectors):
    out_vectors = []
    for vi in vectors:
        ui = 1. * vi
        for uj in out_vectors:
            ui -= (inner(ui, uj) / inner(uj, uj)) * uj
        ui /= norm(ui)
        out_vectors += [ui, ]
    return np.array(out_vectors)

In [ ]:
vectors = np.array([[2.12, 0.75], [0.90, 1.61]])
print(vectors)

In [ ]:
vectors_prime = np.array([[1.77, -0.11], [0.50, 0.15]])
print(vectors_prime)

In [ ]:
# make some events / points -- HACK: weird logic!
rng = np.random.default_rng(21)
points = rng.uniform(size=(31, 2))
points = 2.10 * np.unique(np.round(points, 1), axis=0) + np.array([-0.57, -1.05])[None, :]
print(points.shape)

In [ ]:
def plot_vector(vec, name, **kwargs):
    """
    ## Bugs:
    - Specialized to d=2
    """
    vechat = vec / np.sqrt(np.sum(vec * vec))
    vecorth = np.array([-1. * vechat[1], vechat[0]])
    foo = [[0, vec[i], vec[i] - 0.1 * vechat[i] - 0.02 * vecorth[i], vec[i] - 0.1 * vechat[i] + 0.02 * vecorth[i], vec[i]] for i in [0,1]]
    plt.plot(foo[1], foo[0], **kwargs) # plot 1 before 0
    plt.text(vec[1] + 0.15 * vechat[1], vec[0] + 0.15 * vechat[0], name, ha="center", va="center")
    return None

def plot_vectors(vectors, name_str, grid=True, **kwargs):
    plt.axis("equal")
    plt.axis("off")
    plt.margins(x=0)
    plt.margins(y=0)
    for i, vec in enumerate(vectors):
        print(name_str)
        plot_vector(vec, name_str.format(i), **kwargs)
    if grid:
        plot_grids()
    return None

def plot_grids():
    a = 2.5
    b = (a + 1.) / 2.
    plt.plot([-b, b], [0, 0], "k-", lw=0.5, alpha=0.5, zorder=10)
    plt.plot([0, 0], [-1, a], "k-", lw=0.5, alpha=0.5, zorder=10)
    if np.linalg.slogdet(METRIC)[0] < 0:
        plt.plot([-1, b], [-1, b], "k--", lw=0.5, alpha=0.5, zorder=10)
        plt.plot([1, -b], [-1, b], "k--", lw=0.5, alpha=0.5, zorder=10)
        plt.text(0, a - 0.2, "Lorentz space O(1,1) ", ha="right")
    else:
        plt.text(0, a - 0.2, "Euclidean space O(2) ", ha="right")
    plt.xlim(-b, b)
    plt.ylim(-1., a)
    return None

In [ ]:
plot_vectors(vectors, r"$v_{}$", grid=False, color="k", lw=3, alpha=0.25)
us = orthogonalize(vectors)
plot_vectors(us, r"$\hat u_{}$", color="k", lw=1.5, alpha=1.0)
plt.savefig("E_v.png", dpi=dpi, bbox_inches="tight")

In [ ]:
plot_vectors(vectors_prime, r"$v'_{}$", grid=False, color="r", lw=3, alpha=0.25)
us_prime = orthogonalize(vectors_prime)
plot_vectors(us_prime, r"$\hat u'_{}$", color="r", lw=1.5, alpha=1.0)
plt.savefig("E_vp.png", dpi=dpi, bbox_inches="tight")

In [ ]:
def transform(us, us_prime):
    Q = 0.
    for u, uprime in zip(us, us_prime):
        Q += (np.outer(u, uprime) / inner(uprime, uprime)) @ METRIC
    return Q

def plot_transformed_points(points, vs, vs_prime, **kwargs):
    us = orthogonalize(vs)
    us_prime = orthogonalize(vs_prime)
    points_prime = 1. * points # copy
    Q = transform(us_prime, us)
    for i in range(len(points)):
        points_prime[i] = Q @ points[i]
    plt.scatter(points_prime[:, 1], points_prime[:, 0], **kwargs) # plot 1 before 0
    return None

def plot_transform(points, us, us_prime):
    plt.axis("equal")
    plt.axis("off")
    for i, u in enumerate(us):
        plot_vector(u, r"$\hat u_{}$".format(i), color="k", lw=1.5)
    for i, u in enumerate(us_prime):
        plot_vector(u, r"$\hat u'_{}$".format(i), color="r", lw=1.5)
    plt.scatter(points[:, 1], points[:, 0], c="k", s=3) # plot 1 before 0
    plot_transformed_points(points, us, us_prime, c="r", s=3)
    for f in np.arange(0.1, 0.95, 0.1):
        us_pp = f * us_prime + (1 - f) * us
        plot_transformed_points(points, us, us_pp, c="k", alpha=0.25, s=1)
    plot_grids()
    return None

In [ ]:
plot_transform(points, us, us_prime)
plt.savefig("E_Q.png", dpi=dpi, bbox_inches="tight")

In [ ]:
# Go to Lorentz Group
foo = np.ones(d)
foo[1:] = -1. * foo[1:]
METRIC = np.diag(foo)

In [ ]:
plot_vectors(vectors, r"$v_{}$", grid=False, color="k", lw=3, alpha=0.25)
us = orthogonalize(vectors)
plot_vectors(us, r"$\hat u_{}$", color="k", lw=1.5, alpha=1.0)
plt.savefig("L_v.png", dpi=dpi, bbox_inches="tight")

In [ ]:
plot_vectors(vectors_prime, r"$v'_{}$", grid=False, color="r", lw=3, alpha=0.25)
us_prime = orthogonalize(vectors_prime)
plot_vectors(us_prime, r"$\hat u'_{}$", color="r", lw=1.5, alpha=1.0)
plt.savefig("L_vp.png", dpi=dpi, bbox_inches="tight")

In [ ]:
plot_transform(points, us, us_prime)
plt.savefig("L_Q.png", dpi=dpi, bbox_inches="tight")